# Database Cleaning

In [ ]:
import psycopg2

In [ ]:
conn = psycopg2.connect(database="nutanix", 
                        user="postgres", 
                        host='172.25.221.34',
                        password="Senna",
                        port=1433)

In [ ]:

try:
    cursor = conn.cursor()

    #Delete existing table 
    drop_table_query = "DROP TABLE IF EXISTS ssd_clean_data;"
    cursor.execute(drop_table_query)# Drop the existing table
    conn.commit()  # Commit the drop operation

    # Define the SQL query
    create_table_query = """
        CREATE TABLE ssd_clean_data AS
        SELECT DISTINCT
            p.concord_id,
            p.data_type,
            p.metric,
            p.queue_depth,
            p.num_jobs,
            p.blocksize,
            p.unit,
            p.min_measure,
            p.mean_measure,
            p.median_measure,
            p.max_measure,
            p.stddev_measure,
            d.type AS device_type,
            d.family,
            d.vendor,
            d.model,
            d.firmware,
            d.capacity_GiB,
            d.operating_pci_speed_GTs,
            d.operating_pci_width,
            d.linkrate_Gbs,
            t.name,
            t.reference,
            t.created
        FROM 
            ssd_perf_data p
        JOIN 
            ssd_devices_info d ON p.concord_id = d.concord_id
        JOIN 
            ssd_devices_tests t ON p.concord_id = t.concord_id
        WHERE 
            t.name = 'TEST_NVME_SPDK_PERF';

    """

    # Execute the SQL query
    cursor.execute(create_table_query)

    # Commit the transaction
    conn.commit()

    print("Table 'ssd_clean_data' created successfully.")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()


## Verifies it worked: 

In [ ]:
import psycopg2

# Connect to the database
conn = psycopg2.connect(database="nutanix", 
                        user="postgres", 
                        host='172.25.221.34',
                        password="Senna",
                        port=1433)
field_names = [
        "concord_id",
        "data_type",
        "metric",
        "queue_depth",
        "num_jobs",
        "blocksize",
        "unit",
        "min_measure",
        "mean_measure",
        "median_measure",
        "max_measure",
        "stddev_measure",
        "device_type",
        "family",
        "vendor",
        "model",
        "firmware",
        "capacity_GiB",
        "operating_pci_speed_GTs",
        "operating_pci_width",
        "linkrate_Gbs",
        "name",
        "reference",
        "created"
]

try:
    # Create a cursor
    cur = conn.cursor()

    # Execute a query to count the total number of rows in the ssd_perf_data table
    cur.execute("SELECT COUNT(*) FROM ssd_clean_data;")
    total_rows = cur.fetchone()[0]  # Fetch the count result

    # Print the total number of rows
    print(f"Total number of rows in ssd_clean_data: {total_rows}")
    print(", ".join(field_names))
    # Execute a query to select the first 10 rows from the ssd_perf_data table
    cur.execute("SELECT * FROM ssd_clean_data LIMIT 10;")
    rows = cur.fetchall()  # Fetch all results

    # Print the number of rows returned by the LIMIT query
    print(f"Number of rows returned: {len(rows)}")

    # Print the contents of the table
    for row in rows:
        print(row)  # Print each row

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    if cur:
        cur.close()
    if conn:
        conn.close()
